# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

This notebook teaches MVVM progressively. After each new piece, you run a complete mini app with `root.mainloop()`.

Flow:
1. View only
2. View + ViewModel
3. View + ViewModel + UseCase
4. Swap to another view without changing VM/UseCase

In [ ]:
from pathlib import Path
import sys

# Allow notebook execution from docs/ while importing the project package.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) Step A — View only (no button logic)

The view renders widgets and exposes a `render(...)` method. No business logic yet.

In [ ]:
import tkinter as tk
from tkinter import ttk


class StartViewOnly:
    """Pure View: widgets + render method, no business/use-case logic."""

    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("MVVM Step A: View only")

        # Tkinter variables keep UI labels/progress reactive.
        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=280).pack(padx=8, pady=6)

        # Intentionally no command binding yet.
        ttk.Button(root, text="Start").pack(padx=8, pady=(2, 8), anchor="w")

    def render(self, status_text: str, progress_pct: float) -> None:
        """Render state values to UI controls."""
        self.status_var.set(status_text)
        self.progress_var.set(progress_pct)

In [ ]:
# Step A assembled app (run this cell, close window to continue)
root = tk.Tk()
view = StartViewOnly(root)

# Manual initial paint so learners see where state enters the view.
view.render("View is ready", 0.0)

root.mainloop()

## 2) Step B — Add a ViewModel

Now the view delegates the click to VM. VM updates UI state.

In [ ]:
from dataclasses import dataclass


@dataclass
class StartVmState:
    # Central UI state object owned by the ViewModel.
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False


class StartViewModelStepB:
    """ViewModel: UI state + command handling (still no use case)."""

    def __init__(self):
        self.state = StartVmState()

    def on_start_clicked(self) -> None:
        # Demonstrates that behavior moved out of the View.
        self.state.is_running = True
        self.state.status_text = "Started from ViewModel"
        self.state.progress_pct = 10.0

In [ ]:
class StartViewWithVm:
    """View that delegates events to ViewModel and renders VM state."""

    def __init__(self, root: tk.Tk, vm: StartViewModelStepB):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step B: View + ViewModel")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=280).pack(padx=8, pady=6)

        # View only forwards click to VM.
        ttk.Button(root, text="Start", command=self._on_start).pack(padx=8, pady=(2, 8), anchor="w")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

In [ ]:
# Step B assembled app (run this cell, click Start)
root = tk.Tk()
vm_b = StartViewModelStepB()
view_b = StartViewWithVm(root, vm_b)
root.mainloop()

## 3) Step C — Add a UseCase

UseCase now provides functionality (start + poll progress). VM orchestrates it.

In [ ]:
from seva.adapters.job_rest_mock import JobRestMock


class StartAndPollProgressUseCase:
    """UseCase: starts a demo run via JobRestMock and exposes progress polling."""

    def __init__(self, job_port: JobRestMock):
        # UseCase depends on port implementation (mock here).
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        # Keep notebook small by using the helper typed plan.
        plan = JobRestMock.example_plan()
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        if not self.active_group_id:
            return 0.0

        snapshot = self.job_port.poll_group(self.active_group_id)

        # Demo simplification: read first run progress from first box.
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    """ViewModel that orchestrates use-case calls and updates UI state."""

    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case
        self.state = StartVmState()

    def on_start_clicked(self) -> None:
        group_id = self.use_case.start()
        self.state.is_running = True
        self.state.status_text = f"Started group: {group_id}"

    def on_poll_clicked(self) -> None:
        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress
        self.state.status_text = f"Running... {progress:.1f}%"

        if progress >= 100.0:
            self.state.status_text = "Done"
            self.state.is_running = False

In [ ]:
class StartViewTk:
    """View only: binds buttons to VM commands and renders VM state."""

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step C: View + ViewModel + UseCase")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=280).pack(padx=8, pady=6)

        row = ttk.Frame(root)
        row.pack(padx=8, pady=(2, 8), anchor="w")
        ttk.Button(row, text="Start", command=self._on_start).pack(side="left", padx=(0, 6))
        ttk.Button(row, text="Poll", command=self._on_poll).pack(side="left")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def _on_poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

In [ ]:
# Step C assembled app (run this cell, click Start; progress auto-polls every second)
root = tk.Tk()
uc = StartAndPollProgressUseCase(JobRestMock())
vm_c = StartViewModel(uc)
view_c = StartViewTk(root, vm_c)

def _tick_poll():
    # Polling loop kept in demo runner; View itself stays UI-only.
    if vm_c.state.is_running:
        vm_c.on_poll_clicked()
        view_c.render()
    root.after(1000, _tick_poll)

_tick_poll()
root.mainloop()

## 4) Swap the View (same ViewModel + UseCase)

Now we replace only the View implementation. VM + UseCase remain unchanged.

In [ ]:
class CompactStartViewTk:
    """Alternative View skin with same VM contract (on_start/on_poll + render)."""

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step D: Compact View")

        self.label = ttk.Label(root, text="Idle")
        self.label.pack(padx=8, pady=(8, 4))

        self.progress = ttk.Progressbar(root, maximum=100, length=220)
        self.progress.pack(padx=8, pady=4)

        ttk.Button(root, text="Start", command=self._start).pack(padx=8, pady=2)
        ttk.Button(root, text="Poll", command=self._poll).pack(padx=8, pady=(2, 8))

        self.render()

    def _start(self):
        self.vm.on_start_clicked()
        self.render()

    def _poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.label.configure(text=self.vm.state.status_text)
        self.progress["value"] = self.vm.state.progress_pct

In [ ]:
# Step D assembled app (same VM/UseCase, different View)
root = tk.Tk()
uc_d = StartAndPollProgressUseCase(JobRestMock())
vm_d = StartViewModel(uc_d)
view_d = CompactStartViewTk(root, vm_d)

def _tick_poll_d():
    if vm_d.state.is_running:
        vm_d.on_poll_clicked()
        view_d.render()
    root.after(1000, _tick_poll_d)

_tick_poll_d()
root.mainloop()

## 5) Short adapter cross-link

You can keep View/VM code unchanged and swap adapters behind the use case:
- `JobRestMock()` (offline demo/testing)
- `JobRestAdapter(...)` (real API, e.g., Raspberry Pi host)
- `AmetekJobAdapter` (translate vendor-specific payloads/status to the same `JobPort` method names)

In [ ]:
from seva.domain.ports import JobPort
from seva.domain.entities import ExperimentPlan


class AmetekJobAdapter(JobPort):
    # Example skeleton: translate AMETEK semantics into JobPort contract.
    def start_batch(self, plan: ExperimentPlan):
        print("AMETEK: translate experiment -> vendor payload")
        return ("ametek-demo-group", {})

    def cancel_run(self, box_id, run_id):
        print("AMETEK: cancel run", box_id, run_id)

    def cancel_runs(self, box_to_run_ids):
        print("AMETEK: cancel runs", box_to_run_ids)

    def cancel_group(self, run_group_id):
        print("AMETEK: cancel group", run_group_id)

    def poll_group(self, run_group_id):
        print("AMETEK: poll", run_group_id)
        return {"boxes": {}, "wells": []}

    def download_group_zip(self, run_group_id, target_dir):
        print("AMETEK: download", run_group_id, target_dir)
        return target_dir